## Name Based Country Classification

#### 방법 2: 국적을 랜덤으로 선택한 뒤 그 국적 내 이름을 샘플링하여 학습함

클래스 불균형 문제를 완하하기 위해 국적을 랜덤하게 선택한 후, 국적 내 이름 데이터를 샘플링하여 학습합니다. <br>
(이를 위해 데이터를 국적별로 미리 분류하면 구현이 용이) 
<br>

### One Hot Encoding 을 위해 사용된 문자셋을 얻음

In [1]:
import pandas as pd
from collections import Counter

df = pd.read_csv('name_country.csv')

# 이름 리스트를 얻어옴
name_data = df['Name'].to_list()
# 국적 데이터를 얻어옴 
country_data = df['Country'].to_list()

# 국적 리스트를 구성 : set 을 이용하여 중복 제거 후 정렬
country_list = sorted(set(country_data))
country_count = len(country_list)
print(f"Country Count: {country_count}, Countries={country_list}")

# 국적 to 인덱스로 변환
country_to_index = {country: i for i, country in enumerate(country_list)}
print(f"Country Index: {country_to_index}")

# collections.Counter를 사용하여 국적별 데이터 수 계산 (클래스별 불균형 데이터인지 확인하기 위함)
# country_counts = Counter(country_data)
# print(sorted(country_counts.items()))

Country Count: 18, Countries=['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
Country Index: {'Arabic': 0, 'Chinese': 1, 'Czech': 2, 'Dutch': 3, 'English': 4, 'French': 5, 'German': 6, 'Greek': 7, 'Irish': 8, 'Italian': 9, 'Japanese': 10, 'Korean': 11, 'Polish': 12, 'Portuguese': 13, 'Russian': 14, 'Scottish': 15, 'Spanish': 16, 'Vietnamese': 17}


In [2]:
# 국적별로 데이터들을 구성하는 dict를 생성
# key - country, value - list of names
data_dict = {}
for name, country in zip(name_data, country_data):
  if country not in data_dict:
    data_dict[country] = []
  data_dict[country].append(name)


### Name Character Sets

In [3]:
# one hot encoding 을 위한 문자 집합 생성
unique_chars = set()

# set 집합에 문자열을 추가하면 해당 문자열을 낱개로 쪼개어 각각의 문자들을 하나의 인자로 인식하여 집합에 추가
# 중복된 문자는 추가되지 않음.!!!
for name in name_data:
    unique_chars.update(name)
    if ',' in name:
        print(f"쉼표가 포함된 이름 발견: {name}")

# 문자 집합을 정렬  
unique_chars = sorted(list(unique_chars))
unique_chars = ''.join(unique_chars)
print(f"character count: {len(unique_chars)}, characters={unique_chars}" )

character count: 28, characters= 'abcdefghijklmnopqrstuvwxyz


###  Name to One-Hot Encoded Tensor


In [4]:
import torch

n_letters = len(unique_chars)

def name_to_tensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for i, letter in enumerate(name):
        letter_index = unique_chars.find(letter)
        assert letter_index != -1, "letter not found: " + letter
        tensor[i][letter_index] = 1
    return tensor


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/hyunious/opt/anaconda3/envs/py312/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/hyunious/opt/anaconda3/envs/py312/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/hyunious/opt/anaconda3/envs/py312/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739,

### Create a RNN Model

In [21]:
from xd_rnn import XD_RNN

# 은닉층 수
n_hidden = 32
# 입력층 수, 은닉층 수, 출력층 수
rnn_model = XD_RNN(n_letters, n_hidden, country_count)

# 학습률
learning_rate = 0.0001
# 학습 횟수
iter_count = 200000

# 학습 상태 출력 기준 횟수
print_iter_count = 5000

### Model Trainning

In [19]:
import random
import torch.nn as nn
from torch.optim import Adam

# 최적화 알고리즘
optimizer = Adam(rnn_model.parameters(), lr=learning_rate)

# 손실 함수
loss_fn = nn.CrossEntropyLoss()

# 모델 학습 설정
rnn_model.train()

# 학습 상태 출력 기준 횟수별 loss, predictions
current_loss = 0
correct_predictions = 0

# iteration 만큼 국적별 이름 데이터 학습
for iter_index in range(iter_count):
    # 국적을 랜덤하게 선택
    random_country = random.choice(country_list)
    random_name = random.choice(data_dict[random_country])


    # 데이터 (rows) 학습 
    # 이름을 텐서로 변환 (one-hot encoding)
    input_tensor = name_to_tensor(random_name)
    # 국적을 텐서로 변환
    target_tensor = torch.tensor([country_to_index[random_country]], dtype=torch.long)

    # 모델 은닉층(상태)를 얻어옴
    hidden = rnn_model.get_hidden()

    # 모델 그레디언트 초기화
    rnn_model.zero_grad()

    # rnn 학습
    for char_index in range(input_tensor.size(0)):
        # char tensor 추출 : 2차원 텐서 (1, 28)
        char_tensor = input_tensor[char_index]
        # name char 학습 : 1차원 텐서 (28)
        output, hidden = rnn_model(char_tensor[None, :], hidden)


    # 손실 계산
    loss = loss_fn(output, target_tensor)
    # 손실 역전파
    loss.backward()
    # 최적화 실행
    optimizer.step()

    # 손실 합계 계산
    current_loss += loss.item()

    # 예측 결과 계산
    predicted_index = torch.argmax(output, dim=1)

    # 예측 결과 확인
    correct_predictions += (predicted_index == target_tensor).sum().item()

    # 학습 구간별 학습 상태 출력
    if iter_index % print_iter_count == 0 and iter_index != 0:
        # 평균 손실 계산
        avg_loss = current_loss / print_iter_count
        # 정확도 계산
        accuracy = 100 * correct_predictions / print_iter_count
        # 학습 횟수 출력
        print(f"Iter Index {iter_index}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:2f}%")

        current_loss = 0
        correct_predictions = 0
  
    


Iter Index 5000, Loss: 2.1083, Accuracy: 33.540000%
Iter Index 10000, Loss: 1.6700, Accuracy: 45.460000%
Iter Index 15000, Loss: 1.4931, Accuracy: 50.320000%
Iter Index 20000, Loss: 1.3910, Accuracy: 54.120000%
Iter Index 25000, Loss: 1.2965, Accuracy: 57.340000%
Iter Index 30000, Loss: 1.2422, Accuracy: 59.740000%
Iter Index 35000, Loss: 1.2057, Accuracy: 61.280000%
Iter Index 40000, Loss: 1.1332, Accuracy: 63.220000%
Iter Index 45000, Loss: 1.0644, Accuracy: 65.820000%
Iter Index 50000, Loss: 1.0383, Accuracy: 65.120000%
Iter Index 55000, Loss: 0.9851, Accuracy: 67.460000%
Iter Index 60000, Loss: 0.9709, Accuracy: 68.020000%
Iter Index 65000, Loss: 0.8861, Accuracy: 70.500000%
Iter Index 70000, Loss: 0.8990, Accuracy: 69.740000%
Iter Index 75000, Loss: 0.8753, Accuracy: 70.860000%
Iter Index 80000, Loss: 0.8494, Accuracy: 71.400000%
Iter Index 85000, Loss: 0.8298, Accuracy: 72.720000%
Iter Index 90000, Loss: 0.7719, Accuracy: 73.920000%
Iter Index 95000, Loss: 0.7922, Accuracy: 74.24

### Testing

In [20]:
test_name = 'jinping'
test_tensor = name_to_tensor(test_name)

rnn_model.eval()

hidden = rnn_model.get_hidden()

for char_index in range(test_tensor.size(0)):
    char_tensor = test_tensor[char_index]
    output, hidden = rnn_model(char_tensor[None, :], hidden)

print (f"Output : {output}")

# 예측 결과 확인
predicted_index = torch.argmax(output, dim=1)
print(country_list[predicted_index.item()])


Output : tensor([[-12.3037,  -3.4915,   2.3845,  -0.4657,   5.0800,   1.8066,   4.3514,
         -10.1124,   1.0809,  -6.1061,  -7.9661,  -1.6875,   3.8098, -11.9028,
           3.0550,   1.1833,  -4.3765,  -4.1357]], grad_fn=<AddmmBackward0>)
English
